<a href="https://colab.research.google.com/github/harshankbansal/pytorch-deep-learning/blob/main/lectures/07_experimentations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tracking Experiments

With ML projects, we usually train 20+ models, and which introduces a different set of problems. Tracking of these experiments is important and there are various tools just for this

1. In built pytorch's board
2. 3rd party tools like weights and bias https://wandb.ai/site/
3. mlflow - another 3rd party

there are many more solutsion

In [1]:
# Set up imports. Thanks to python it is unnecessarily complicated
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if not IN_COLAB:
    try:
        from going_modular.going_modular import data_setup, engine
    except ImportError:
        import os
        import sys
        from pathlib import Path
        sys.path.append(str(Path(os.path.abspath('')).parent))
        from going_modular.going_modular import data_setup
else:
    try:
        from going_modular.going_modular import data_setup, engine
    except:
        # Get the going_modular scripts
        print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
        !git clone https://github.com/harshankbansal/pytorch-deep-learning
        !mv pytorch-deep-learning/going_modular .
        !mv pytorch-deep-learning/helper_functions.py .
        !mv pytorch-deep-learning/data .
        !rm -rf pytorch-deep-learning
        from going_modular.going_modular import data_setup, engine

import helper_functions

[INFO] Couldn't find going_modular scripts... downloading them from GitHub.
Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 4364, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 4364 (delta 16), reused 15 (delta 15), pack-reused 4343 (from 2)
Receiving objects: 100% (4364/4364), 762.51 MiB | 33.43 MiB/s, done.
Resolving deltas: 100% (2645/2645), done.
Updating files: 100% (257/257), done.
